In [1]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")



from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [2]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to Tamil and Telugu. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="Here's the translation:\n\n*   **Tamil:** நான் புரோகிராமிங்கை விரும்புகிறேன். (Naan programming-ai virumbugiren.)\n*   **Telugu:** నేను ప్రోగ్రామింగ్ని ప్రేమిస్తున్నాను. (Nenu programming-ni premistunnanu.)", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-lite', 'safety_ratings': []}, id='run--471c7c5c-4127-41dc-a2a4-62bb4306c7ea-0', usage_metadata={'input_tokens': 22, 'output_tokens': 79, 'total_tokens': 101, 'input_token_details': {'cache_read': 0}})

# 1. Simple Chain

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser


prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a facts expert who knows facts about {animal}."),
        ("human", "Tell me {fact_count} facts."),
    ]
)

# Create the combined chain using LangChain Expression Language (LCEL)
chain = prompt_template | llm | StrOutputParser()
# chain = prompt_template | llm

# Run the chain
result = chain.invoke({"animal": "elephant", "fact_count": 1})

# Output
print(result)

Elephants are the largest land animals on Earth.


# 2. Chains Internal Working

In [4]:
from langchain.schema.runnable import RunnableLambda, RunnableSequence
from langchain.prompts import ChatPromptTemplate


prompt_template = ChatPromptTemplate.from_messages(
     [
        ("system", "You love facts and you tell facts about {animal}"),
        ("human", "Tell me {count} facts."),
    ]
)

# Create individual runnables (steps in the chain)
format_prompt = RunnableLambda(lambda x: prompt_template.format_prompt(**x))
invoke_model = RunnableLambda(lambda x: llm.invoke(x.to_messages()))
parse_output = RunnableLambda(lambda x: x.content)

# Create the RunnableSequence (equivalent to the LCEL chain)
chain = RunnableSequence(first=format_prompt, middle=[invoke_model], last=parse_output)

# Run the chain
response = chain.invoke({"animal": "cat", "count": 2})

# Output
print(response)

Alright, here are two purr-fect facts about cats:

1.  **Cats can jump up to six times their height!** That's like a human leaping over a small car!
2.  **A cat's nose is as unique as a human fingerprint.** No two cats have the same pattern of ridges and bumps on their nose.


## Option 1: Use LCEL (Recommended)
The pipe operator (|) is much cleaner for multiple components:

In [5]:
from langchain.schema.runnable import RunnableLambda
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You love facts and you tell facts about {animal}"),
    ("human", "Tell me {count} facts."),
])

# Create individual runnables
format_prompt = RunnableLambda(lambda x: prompt_template.format_prompt(**x))
invoke_model = RunnableLambda(lambda x: llm.invoke(x.to_messages()))
add_metadata = RunnableLambda(lambda x: {"content": x.content, "timestamp": "2024"})
extract_content = RunnableLambda(lambda x: x["content"])
uppercase_output = RunnableLambda(lambda x: x.upper())

# Chain them with LCEL - much cleaner!
chain = format_prompt | invoke_model | add_metadata | extract_content | uppercase_output

response = chain.invoke({"animal": "cat", "count": 2})
print(response)

ALRIGHT, HERE ARE TWO PURR-FECT FACTS ABOUT CATS:

1.  **CATS CAN JUMP UP TO SIX TIMES THEIR HEIGHT!** THAT'S LIKE A HUMAN LEAPING OVER A SMALL CAR!
2.  **A CAT'S NOSE IS AS UNIQUE AS A HUMAN FINGERPRINT.** NO TWO CATS HAVE THE SAME PATTERN OF RIDGES AND BUMPS ON THEIR NOSE.


## Option 2: RunnableSequence with All Components in Middle
Put all intermediate steps in the middle list:

In [6]:
from langchain.schema.runnable import RunnableSequence

# All intermediate steps go in middle
chain = RunnableSequence(
    first=format_prompt,
    middle=[invoke_model, add_metadata, extract_content],  # Multiple components here
    last=uppercase_output
)

response = chain.invoke({"animal": "cat", "count": 2})
print(response)

ALRIGHT, HERE ARE TWO PURR-FECT FACTS ABOUT CATS:

1.  **CATS CAN JUMP UP TO SIX TIMES THEIR HEIGHT!** THAT'S LIKE A HUMAN LEAPING OVER A CAR!
2.  **A CAT'S NOSE IS AS UNIQUE AS A HUMAN FINGERPRINT.** NO TWO CATS HAVE THE SAME PATTERN OF RIDGES AND BUMPS ON THEIR NOSE.


## Option 3: Create from List
You can create a sequence from a list of runnables:

In [9]:
from langchain.schema.runnable import RunnableSequence

# Create from list
components = [
    format_prompt,
    invoke_model, 
    add_metadata,
    extract_content,
    uppercase_output
]

chain = RunnableSequence(*components)

response = chain.invoke({"animal": "cat", "count": 2})
print(response)

ALRIGHT, HERE ARE TWO PURR-FECT FACTS ABOUT CATS:

1.  **CATS CAN JUMP UP TO SIX TIMES THEIR HEIGHT.** THAT'S LIKE A HUMAN LEAPING OVER A CAR!
2.  **A CAT'S NOSE IS AS UNIQUE AS A HUMAN FINGERPRINT.** NO TWO CATS HAVE THE SAME NOSE PATTERN.


# Sequential Chains



In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda


# Define prompt templates
animal_facts_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You like telling facts and you tell facts about {animal}."),
        ("human", "Tell me {count} facts."),
    ]
)

# Define a prompt template for translation to French
translation_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a translator and convert the provided text into {language}."),
        ("human", "Translate the following text to {language}: {text}"),
    ]
)

# Define additional processing steps using RunnableLambda
count_words = RunnableLambda(lambda x: f"Word count: {len(x.split())}\n{x}")
prepare_for_translation = RunnableLambda(lambda output: {"text": output, "language": "Dogri"})


# Create the combined chain using LangChain Expression Language (LCEL)
chain = animal_facts_template | llm | StrOutputParser() | prepare_for_translation | translation_template | llm | StrOutputParser() 

# Run the chain
result = chain.invoke({"animal": "cat", "count": 2})

# Output
print(result)

ठीक ऐ, एत्थें बिल्लियां बारै दो बड़िया गल्लें न:

1.  **बिल्ली अपनी ऊँचाई थमां छै गणा उप्पर छलांग ला सकदी ऐ.** एह् इक मानुख दे इक छोटी गाड़ी उप्पर छलांग लाने आं ऐ!
2.  **इक बिल्ली दी नाक इक मानुख दी उंगली दे निशानें जियां खास ऐ.** दो बिल्लियां दी नाक दा पैटर्न इक जियां नेईं होंदा।


# Parallel Chains
---

In [13]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableLambda, RunnableParallel
from langchain.schema.output_parser import StrOutputParser

# Define prompt template for movie summary
summary_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie critic."),
        ("human", "Provide a brief summary of the movie {movie_name}."),
    ]
)

# Define plot analysis step
def analyze_plot(plot):
    plot_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a movie critic."),
            ("human", "Analyze the plot: {plot}. What are its strengths and weaknesses?"),
        ]
    )
    return plot_template.format_prompt(plot=plot)

# Define character analysis step
def analyze_characters(characters):
    character_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a movie critic."),
            ("human", "Analyze the characters: {characters}. What are their strengths and weaknesses?"),
        ]
    )
    return character_template.format_prompt(characters=characters)

# Combine analyses into a final verdict
def combine_verdicts(plot_analysis, character_analysis):
    return f"Plot Analysis:\n{plot_analysis}\n\nCharacter Analysis:\n{character_analysis}"

# Simplify branches with LCEL
plot_branch_chain = (
    RunnableLambda(lambda x: analyze_plot(x)) | llm | StrOutputParser()
)

character_branch_chain = (
    RunnableLambda(lambda x: analyze_characters(x)) | llm | StrOutputParser()
)

# Create the combined chain using LangChain Expression Language (LCEL)
chain = (
    summary_template
    | llm
    | StrOutputParser()
    | RunnableParallel(branches={"plot": plot_branch_chain, "characters": character_branch_chain})
    | RunnableLambda(lambda x: combine_verdicts(x["branches"]["plot"], x["branches"]["characters"]))
)

# Run the chain
result = chain.invoke({"movie_name": "Inception"})

print(result)

Plot Analysis:
Alright, let's dive into the dreamscape and dissect Christopher Nolan's "Inception."

**Strengths:**

*   **Concept and Originality:** "Inception" is undeniably a triumph of imagination. The core concept of dream-sharing, dream layering, and the manipulation of the subconscious is incredibly inventive. It's a high-concept thriller that immediately grabs your attention and keeps you hooked with its unique premise.
*   **Visual Spectacle:** Nolan is a master of visual storytelling, and "Inception" is a feast for the eyes. The dream sequences are breathtaking, from the zero-gravity fight scenes to the collapsing cityscapes. The film utilizes practical effects and CGI seamlessly, creating a truly immersive and unforgettable experience. The visual language of the dream worlds is distinct and memorable.
*   **Intricate Plot and Pacing:** The film's complex plot, while challenging, is also its strength. The multiple layers of dreams, the rules of dream physics, and the ticking 

In [14]:
from IPython.display import Markdown
Markdown(result)

Plot Analysis:
Alright, let's dive into the dreamscape and dissect Christopher Nolan's "Inception."

**Strengths:**

*   **Concept and Originality:** "Inception" is undeniably a triumph of imagination. The core concept of dream-sharing, dream layering, and the manipulation of the subconscious is incredibly inventive. It's a high-concept thriller that immediately grabs your attention and keeps you hooked with its unique premise.
*   **Visual Spectacle:** Nolan is a master of visual storytelling, and "Inception" is a feast for the eyes. The dream sequences are breathtaking, from the zero-gravity fight scenes to the collapsing cityscapes. The film utilizes practical effects and CGI seamlessly, creating a truly immersive and unforgettable experience. The visual language of the dream worlds is distinct and memorable.
*   **Intricate Plot and Pacing:** The film's complex plot, while challenging, is also its strength. The multiple layers of dreams, the rules of dream physics, and the ticking clock create a constant sense of tension and urgency. Nolan masterfully weaves together the different dream levels, keeping the audience engaged and guessing. The pacing is generally excellent, with well-placed action sequences and moments of quiet reflection.
*   **Strong Performances:** The cast is superb. Leonardo DiCaprio delivers a compelling performance as Cobb, burdened by guilt and driven by a desire to reunite with his children. The supporting cast, including Joseph Gordon-Levitt, Elliot Page, Tom Hardy, and Cillian Murphy, are all excellent, each bringing their own unique skills and personalities to the team.
*   **Thematic Depth:** Beyond the action and spectacle, "Inception" explores profound themes. It delves into the nature of reality, the power of the subconscious, the impact of guilt and regret, and the complexities of human relationships. The film encourages viewers to question their own perceptions and consider the fragility of the mind.
*   **Sound Design and Score:** Hans Zimmer's score is iconic and integral to the film's success. It's both epic and haunting, perfectly complementing the visuals and heightening the emotional impact of key scenes. The sound design is also exceptional, creating a sense of immersion and amplifying the tension.

**Weaknesses:**

*   **Complexity and Exposition:** The film's intricate plot can be overwhelming at times. The sheer number of rules, dream levels, and technical jargon can be difficult to grasp on a single viewing. The film relies heavily on exposition, with characters often explaining the mechanics of dream-sharing, which can sometimes feel clunky.
*   **Emotional Resonance:** While the film touches on emotional themes, some viewers may find it difficult to fully connect with the characters on an emotional level. The focus on the heist and the technical aspects of dream manipulation can sometimes overshadow the characters' personal struggles. Cobb's motivations, while understandable, can feel somewhat underdeveloped.
*   **Plot Holes and Ambiguity:** Despite its meticulous construction, "Inception" has a few plot holes and ambiguities that can be frustrating for some viewers. The ending, in particular, is intentionally ambiguous, leaving the audience to interpret whether Cobb is truly back in reality. This can be a strength for some, but a weakness for others who prefer a more definitive resolution.
*   **Pacing Issues (in certain sections):** While the overall pacing is good, some sections, particularly the exposition-heavy scenes, can feel a bit slow. The film's length (over two hours) can also be a factor for some viewers.
*   **Reliance on Action:** While the action sequences are visually stunning, the film sometimes relies too heavily on them to drive the plot forward. The focus on the heist can overshadow the more subtle and nuanced exploration of the film's themes.

**Overall:**

"Inception" is a groundbreaking and ambitious film that pushes the boundaries of cinematic storytelling. Its strengths lie in its originality, visual spectacle, intricate plot, and thematic depth. While its complexity, occasional plot holes, and reliance on exposition can be weaknesses, the film's overall impact and lasting legacy are undeniable. It's a film that rewards repeat viewings and continues to spark discussion and debate long after the credits roll. It's a must-see for any fan of science fiction, action, or thought-provoking cinema.

Character Analysis:
Alright, let's dive into the characters of "Inception" and dissect their strengths, weaknesses, and the fascinating ways they contribute to this mind-bending heist.

**Dom Cobb (Leonardo DiCaprio):**

*   **Strengths:**
    *   **Master Architect & Dream Weaver:** Cobb is the undisputed leader, the architect of the dream worlds. He possesses an unparalleled understanding of dream mechanics, allowing him to manipulate and control the environment. He can build intricate, believable dreamscapes.
    *   **Strategic Genius:** Cobb is a brilliant strategist, meticulously planning every detail of the inception. He anticipates potential problems and adapts to the unpredictable nature of the dream worlds.
    *   **Experienced & Resourceful:** Years of experience in dream extraction have honed his skills and provided him with a network of contacts and resources. He knows how to navigate the dangers of the subconscious.
    *   **Driven by Love (and Guilt):** Cobb's primary motivation is to return to his children. This love fuels his determination and pushes him to take on the impossible task.
*   **Weaknesses:**
    *   **Haunted by Mal:** Cobb is deeply tormented by the projection of his deceased wife, Mal. She manifests in the dream worlds, sabotaging his plans and representing his guilt and unresolved trauma. This makes him emotionally vulnerable and a liability in the dream.
    *   **Emotional Instability:** His grief and guilt make him prone to emotional outbursts and impulsive actions, which can jeopardize the mission.
    *   **Difficulty Letting Go:** Cobb struggles to accept the reality of Mal's death and the consequences of his past actions. This inability to move on hinders his ability to focus and make clear decisions.
    *   **Reliance on Others:** While a leader, Cobb is reliant on his team. He needs their expertise to succeed, and their failures directly impact him.

**Arthur (Joseph Gordon-Levitt):**

*   **Strengths:**
    *   **The Point Man:** Arthur is Cobb's right-hand man, the meticulous planner and strategist who handles the logistics and security of the dream heists. He's the one who researches the target, plans the dream architecture, and ensures the team's safety.
    *   **Level-Headed & Pragmatic:** Arthur is the voice of reason, the one who keeps the team grounded and focused. He's less emotionally driven than Cobb, making him a valuable asset in high-pressure situations.
    *   **Resourceful & Adaptable:** Arthur is quick on his feet and able to improvise when things go wrong. He can think on his feet and find solutions to unexpected problems.
    *   **Loyal & Reliable:** Arthur is fiercely loyal to Cobb and the team. He's willing to put himself in harm's way to protect them.
*   **Weaknesses:**
    *   **Over-Reliance on Planning:** Arthur's meticulous planning can sometimes lead to rigidity. He can struggle to adapt when the unexpected happens, and he can be slow to react to changes in the dream.
    *   **Limited Emotional Range:** While his pragmatism is a strength, it can also make him seem detached and less empathetic.
    *   **Secondary Role:** Arthur is always in the shadow of Cobb. He is not the leader, and his role is to support Cobb's vision.

**Ariadne (Elliot Page):**

*   **Strengths:**
    *   **Dream Architect:** Ariadne is the newest member of the team, a brilliant architecture student who designs the dream worlds. She has a natural talent for creating complex and believable dreamscapes.
    *   **Creative & Innovative:** Ariadne brings a fresh perspective to the team, challenging Cobb's established methods and pushing the boundaries of dream architecture.
    *   **Quick Learner:** She quickly grasps the intricacies of dream sharing and adapts to the dangers of the subconscious.
    *   **Empathy & Insight:** Ariadne is able to understand and empathize with Cobb's emotional struggles, helping him to confront his demons.
*   **Weaknesses:**
    *   **Inexperience:** As the newest member, Ariadne lacks the experience of the other team members. She is still learning the ropes and can be overwhelmed by the complexity of the dream worlds.
    *   **Vulnerability:** Her inexperience makes her more susceptible to the dangers of the subconscious, particularly the projections of the target and Cobb's Mal.
    *   **Emotional Involvement:** Ariadne becomes emotionally invested in Cobb's well-being, which can cloud her judgment and make her more vulnerable.

**Eames (Tom Hardy):**

*   **Strengths:**
    *   **The Forger:** Eames is a master forger, able to impersonate people within the dream and manipulate their actions. He is a chameleon, able to adapt to any situation.
    *   **Charismatic & Resourceful:** Eames is a charming and resourceful individual who can talk his way out of any situation. He is a valuable asset in the dream world.
    *   **Expert in Combat:** Eames is skilled in combat and can handle himself in a fight. He is a valuable asset in the dream world.
    *   **Witty & Entertaining:** Eames provides comic relief and helps to lighten the mood in tense situations.
*   **Weaknesses:**
    *   **Unpredictable:** Eames is a bit of a wildcard, and his actions can sometimes be unpredictable. He is not always reliable.
    *   **Self-Serving:** Eames is primarily motivated by his own self-interest. He is not always willing to put himself in harm's way for the sake of the mission.
    *   **Overconfident:** Eames can be overconfident in his abilities, which can lead to mistakes.

**Yusuf (Dileep Rao):**

*   **Strengths:**
    *   **The Chemist:** Yusuf is the chemist who provides the sedatives that allow the team to share dreams. He is essential to the success of the mission.
    *   **Practical & Resourceful:** Yusuf is a practical and resourceful individual who can find solutions to any problem.
    *   **Calm Under Pressure:** Yusuf is able to remain calm under pressure, which is essential in the dream world.
*   **Weaknesses:**
    *   **Limited Combat Skills:** Yusuf is not skilled in combat and is vulnerable in the dream world.
    *   **Limited Experience:** Yusuf is not as experienced as the other team members.
    *   **Relatively Passive Role:** Yusuf's role is primarily supportive, and he doesn't have as much agency in the dream world as the other team members.

**Saito (Ken Watanabe):**

*   **Strengths:**
    *   **The Client:** Saito is the wealthy businessman who hires Cobb for the inception. He is the driving force behind the mission.
    *   **Determined & Ruthless:** Saito is determined to succeed and is willing to do whatever it takes to achieve his goals.
    *   **Resourceful:** Saito has access to vast resources and can provide the team with the tools and support they need.
*   **Weaknesses:**
    *   **Vulnerable:** Saito is the target of the inception, and he is vulnerable to the team's manipulations.
    *   **Demanding:** Saito is demanding and expects the team to succeed.
    *   **Unpredictable:** Saito's actions can sometimes be unpredictable.

**Mal (Marion Cotillard):**

*   **Strengths:**
    *   **The Projection:** Mal is a powerful projection of Cobb's subconscious, representing his guilt and unresolved trauma. She is a formidable opponent in the dream worlds.
    *   **Intelligent & Manipulative:** Mal is intelligent and manipulative, and she is able to exploit Cobb's weaknesses.
*   **Weaknesses:**
    *   **Unstable:** Mal is unstable and unpredictable, and she can sabotage the team's plans.
    *   **Driven by Grief:** Mal is driven by her grief and her desire to be with Cobb.
    *   **A Manifestation of Cobb's Guilt:** Mal is not a real person, and she is a manifestation of Cobb's guilt and unresolved trauma.

In essence, "Inception" is a story about a team of specialists with unique skills, each with their own strengths and weaknesses. The success of the mission hinges on their ability to work together, overcome their individual flaws, and navigate the treacherous landscape of the human subconscious. The film's brilliance lies in how it uses these characters to explore complex themes of reality, memory, and the power of the human mind.

# Conditional Chains
---

In [18]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableBranch


positive_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a customer service bot. Keep responses brief and professional. Write a short thank you note (maximum 2 sentences)."),
        ("human",
         "Generate a thank you note for this positive feedback: {feedback}."),
    ]
)

negative_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a customer service bot. Keep responses brief and professional. Acknowledge the issue and offer next steps (maximum 3 sentences)."),
        ("human",
         "Generate a response addressing this negative feedback: {feedback}."),
    ]
)

neutral_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a customer service bot. Keep responses brief and professional. Ask for specific details (maximum 2 sentences)."),
        (
            "human",
            "Generate a request for more details for this neutral feedback: {feedback}.",
        ),
    ]
)

escalate_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a customer service bot. Keep responses brief and professional. Inform about escalation to human agent (maximum 2 sentences)."),
        (
            "human",
            "Generate a message to escalate this feedback to a human agent: {feedback}.",
        ),
    ]
)

classification_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human",
         "Classify the sentiment of this feedback as positive, negative, neutral, or escalate. Return ONLY the classification word: {feedback}."),
    ]
)

branches = RunnableBranch(
    (
        lambda x: "positive" in x["classification"].lower(),
        positive_feedback_template | llm | StrOutputParser()
    ),
    (
        lambda x: "negative" in x["classification"].lower(),
        negative_feedback_template | llm | StrOutputParser()
    ),
    (
        lambda x: "neutral" in x["classification"].lower(),
        neutral_feedback_template | llm | StrOutputParser()
    ),
    escalate_feedback_template | llm | StrOutputParser()
)

classification_chain = classification_template | llm | StrOutputParser()

def combine_classification_with_feedback(input_data):
    classification = classification_chain.invoke(input_data)
    return {
        "feedback": input_data["feedback"],
        "classification": classification
    }

chain = combine_classification_with_feedback | branches

review = "The product is terrible. It broke after just one use and the quality is very poor."
result = chain.invoke({"feedback": review})

print(result)

I'm sorry to hear about your experience with our product. Please contact our customer service team so we can arrange a replacement or issue a refund. We appreciate your feedback and are committed to improving our quality.
